# DS-SF-36 | 14 | Trees | Codealong | Starter Code

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import math

## Part A | The 2008 Democratic Primaries

(dataset adapted from http://www.stat.ucla.edu/~cocteau/primaries.csv)

In [49]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-14-2008-democrat-primaries.csv'))

In [50]:
df.columns

Index([u'fips', u'county_name', u'state_postal', u'region', u'election_date',
       u'racetype', u'tvotes', u'clinton', u'obama', u'edwards', u'margin',
       u'winner', u'POP05_SQMI', u'popUnder30_00', u'pop65up_00',
       u'presVote04', u'kerry04', u'Bush04', u'pres04margin', u'pres04winner',
       u'pop06', u'pop00', u'hisp06', u'white06', u'black06', u'indian06',
       u'asian06', u'hawaii06', u'mixed06', u'pct_less_30k', u'pct_more_100k',
       u'pct_hs_grad', u'pct_labor_force', u'pct_homeowner', u'unempFeb07',
       u'unempFeb08', u'unempChg', u'pctUnins00', u'subForPctHomes',
       u'poverty05', u'median_hhi05', u'Catholic', u'So.Bapt.Conv',
       u'Un.Methodist', u'E.L.C.A.', u'Construction', u'Manufacturing',
       u'FinancialActivities', u'GoodsProducing', u'ServiceProviding'],
      dtype='object')

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2261 entries, 0 to 2260
Data columns (total 50 columns):
fips                   2261 non-null int64
county_name            2261 non-null object
state_postal           2261 non-null object
region                 2261 non-null object
election_date          2261 non-null object
racetype               2261 non-null object
tvotes                 2261 non-null int64
clinton                2261 non-null int64
obama                  2261 non-null int64
edwards                2261 non-null int64
margin                 2261 non-null float64
winner                 2241 non-null object
POP05_SQMI             2260 non-null float64
popUnder30_00          2260 non-null float64
pop65up_00             2260 non-null float64
presVote04             2260 non-null float64
kerry04                2260 non-null float64
Bush04                 2260 non-null float64
pres04margin           2260 non-null float64
pres04winner           2260 non-null object
pop06     

In [52]:
df.winner.unique()

array(['obama', 'clinton', nan], dtype=object)

In [53]:
df.winner.value_counts(dropna=False)

clinton    1210
obama      1031
NaN          20
Name: winner, dtype: int64

In [54]:
df['c'] = (df.winner =='obama') *1.

In [55]:
df.pop06

0        49730.0
1       169162.0
2        28171.0
3        21482.0
4        56436.0
          ...   
2256     38763.0
2257     19288.0
2258     20213.0
2259      7819.0
2260      6762.0
Name: pop06, Length: 2261, dtype: float64

In [56]:
df['pct_black06'] = df.black06/df.pop06

In [57]:
parent_df=df

In [58]:
left_child_df = parent_df[parent_df.pct_black06 <= 0.20]

In [59]:
right_child_df = parent_df[parent_df.pct_black06 > 0.20]

# Method 02
#right_child_df = parent_df.drop(left_child_df.index)

### First cut: Is a county more than 20% black?

In [60]:
right_child_df.shape[0]

450

In [61]:
left_child_df.shape[0]

1810

Ans: No

#### First cut/right node

In [62]:
# TODO

In [77]:
def obama_vs_clinton(df):
    obama = (df.c == 1).sum()
    clinton = (df.c == 0).sum()
    if obama > clinton:
        print 'Obama wins these counties {0} to {1}.'.format(obama, clinton)
    elif clinton > obama:
        print 'Clinton wins these counties {} to {}.'.format(clinton, obama)
    else:
        print 'Obama and Clinton tie in these counties {} {}.'.format(obama, clinton)

In [64]:
obama_vs_clinton(right_child_df)

Obama wins these counties 380 to 70.


### Second cut: Is high school graduation rate higher than 78%?

In [65]:
parent_df  = left_child_df

In [66]:
left_child_df = parent_df[parent_df.pct_hs_grad <= 0.78]

In [67]:
right_child_df = parent_df.drop(left_child_df.index)

In [69]:
obama_vs_clinton(left_child_df)

Clinton wins these counties 714 to 93.


### Third cut: Is high school graduation rate higher than 87%?

In [71]:
parent_df= right_child_df

In [72]:
left_child_df = parent_df[parent_df.pct_hs_grad <.87]

In [73]:
right_child_df = parent_df.drop(left_child_df.index)

In [78]:
obama_vs_clinton(right_child_df)

Obama wins these counties 177 to 36.


## Part B | Building the 2008 Democratic Primaries Decision Tree by Hand

In [ ]:
class Node:

    @staticmethod
    def root(root_df):
        cs = sorted(set(root_df.c))
        return Node(cs, root_df)

    def decision(self, left_filter):
        # Collect the observations for which the decision split is true and
        # create the corresponding left node

        left_filter = left_filter(self.df)
        left_df = self.df[left_filter]
        self.left = Node(self.cs, left_df)

        # Same thing on the right side but for the observations that don't
        # satisfy the decision split (the "else")

        right_df = self.df.drop(left_df.index)
        self.right = Node(self.cs, right_df)

        # The entropy after the decision split is the weighted average of the
        # children entropy

        self.after = (self.left.samples * self.left.before
                      + self.right.samples * self.right.before) / self.samples

        # The information gain corresponds to the entropy lost between the
        # parent node (this node and the "before") and its child (the "after")

        self.information_gain = self.before - self.after

        return self

    def __init__(self, cs, df):
        self.cs = cs
        self.df = df

        # Counts of the remaining observations in the subspace per classes
        self.counts = [(self.df.c == c).sum() for c in self.cs]

        # Number of observations in the subspace
        self.samples = sum(self.counts)

        # For empty subspaces, probabilties and entropy are set to zero
        if self.samples == 0:
            self.probabilities = [.0 for count in self.counts]
            self.before = .0
        else:
            self.probabilities = [1. * count / self.samples for count in self.counts]
            self.before = - sum(map(lambda p: p * math.log(p, 2),
                                    filter(lambda p : p > .0, self.probabilities)))

    def status(self):
        print 'classes                       =', self.cs
        print 'before:'
        print "\tparent:"
        print "\t\tsamples       =", self.samples
        print "\t\tcounts        =", self.counts
        print "\t\tprobabilities =", self.probabilities
        print "\t\tentropy       =", self.before
        print 'after:'
        print "\tleft child:"
        print "\t\tsamples       =", self.left.samples
        print "\t\tcounts        =", self.left.counts
        print "\t\tprobabilities =", self.left.probabilities
        print "\t\tentropy       =", self.left.before
        print "\tright child:"
        print "\t\tsamples       =", self.right.samples
        print "\t\tcounts        =", self.right.counts
        print "\t\tprobabilities =", self.right.probabilities
        print "\t\tentropy       =", self.right.before
        print
        print 'before entropy                =', self.before
        print 'after entropy                 =', self.after
        print 'information gain              =', self.information_gain

In [ ]:
df.c = df.winner

### First cut

In [ ]:
# TODO

#### Candidate #1: Is a county more than 20% black?

In [ ]:
# TODO

#### Candidate #2: Is high school graduation rate higher than 78%?

In [ ]:
# TODO

#### Candidate #3: Is high school graduation rate higher than 87%?

In [ ]:
# TODO

### Second cut

In [ ]:
# TODO

### Third cut

In [ ]:
# TODO